In [7]:
# This program parses the log file from the mqtt logger for ICSPACE20


In [20]:
# Parse line

import json

time_stamp_info_part_size = len("2020-04-18 22:18:57,348 [INFO] b'")

def parse_line(line:str):
    line = line[time_stamp_info_part_size:]
    line = line[:-2]
    
    json_line = json.loads(line)
    #print(line)
    print(json_line)
    

In [21]:
f = open("mqtt.log", "r")
for line in f:
  parse_line(line)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'app_id': 'icss_lora_tracker', 'dev_id': 'icspace20', 'hardware_serial': '0087FA6DB8EAAF57', 'port': 99, 'counter': 1744, 'is_retry': True, 'payload_raw': 'AHMVRgFn9c0CiAf4/gBm0Q7wOAMCAYsEAQk=', 'payload_fields': {'analog_in_3': 3.95, 'barometric_pressure_0': 544.6, 'digital_out_4': 9, 'gps_2': {'altitude': 9790, 'latitude': 52.2494, 'longitude': 2.6321}, 'temperature_1': -261.1}, 'metadata': {'time': '2020-08-22T14:52:58.195035474Z', 'frequency': 868.1, 'modulation': 'LORA', 'data_rate': 'SF8BW125', 'airtime': 154112000, 'coding_rate': '4/5', 'gateways': [{'gtw_id': 'eui-000080029c10db9b', 'timestamp': 3732521220, 'time': '2020-08-22T14:52:57.684581Z', 'channel': 5, 'rssi': -119, 'snr': -2.2, 'rf_chain': 0, 'latitude': 52.41427, 'longitude': 4.82986}, {'gtw_id': 'eui-40d63cfffe022068', 'timestamp': 744764252, 'time': '', 'channel': 0, 'rssi': -118, 'snr': -3.8, 'rf_chain': 0}, {'gtw_id': 'eui-0000024b08031d2d', 'timestamp': 2632000956, 'time': '2020-08-22T14:52:57.684481Z', 'channel'